    ##338

##339

In [ ]:
#Given an array of numbers and a number k,
#determine if there are three entries in the array which add up to the specified number k.

#Given [20, 303, 3, 4, 25] and k = 49, return true as 20 + 4 + 25 = 49

In [4]:
#1. loop through all possible triplets, comparing the sum to the number k provided?
#O(n^3) [super inefficient]

def triplet_sums_to_k(arr, k):
    for x in range(len(arr)):
        for y in range(x + 1, len(arr)):
            for z in range(y + 1, len(arr)):
                if arr[x] + arr[y] + arr[z] == k:
                    return True
    return False


arr1 = [20, 303, 3, 4, 25]
triplet_sums_to_k(arr1, 40) #False
triplet_sums_to_k(arr1, 49) #True

True

In [ ]:
#2. A Hash table (a slightly better)
#After fixing the first number of the triplet,
#we find whether or not there are two other elements that sum to the remainder.

#O(n^2)
#We use an additional O(n) space for the hash table.

def triplet_sums_to_k(arr, k):
    def pair_sums_to_k(arr, k):
        num_set = set()
        for n in arr:
            if k - n in num_set:
                return True
            num_set.add(n)
        return False
    
    for i in range(len(arr) - 2):
        if pair_sums_to_k(arr[i + 1:], k - arr[i]):
            return True
        
    return False

In [10]:
#3. We can sort the array
#Then, for each entry in tha array, we find whether or not there are two other elements that sum to the remainder.

#Since the array is sorted (important)
#we can write a function that solves this 2-sum sub-problem in O(n) time.
#[Two-pointer technique]

#Overall O(n^2)

def triplet_sums_to_k(arr, k):
    def pair_sums_to_k(arr, k, start):
        lo = start
        hi = len(arr) - 1
        while lo < hi:
            if arr[lo] + arr[hi] == k:
                return True
            elif arr[lo] + arr[hi] < k:
                lo += 1
            else:
                hi -= 1
        return False
    
    arr.sort()
    for i in range(len(arr) - 2):
        if pair_sums_to_k(arr, k - arr[i], i + 1):
            return True
        
    return False

x = [30, 2, 204, 6, 26]
triplet_sums_to_k(x, 261)

False

##341

In [ ]:
#google

#You are given an N by N matrix of random letters and a dictionary of words.
#Find the maximum number of words that can be packed on the board from the given dictionary.

#A word is considered to be able to be packed on the board if:
# - It can be found in the dictionary
# - It can be constructed from untaken letters by other words found so far on the board
# - The letters are adjacent to each other (vertically and horizontally, not diagonally)

#Each tile can be visited only once by any word.
#{ 'eat', 'rain', 'in', 'rat' }

#and matrix:
#[['e', 'a', 'n'],
#['t', 't', 'i'],
#['a', 'r', 'a']]

#Your function should return 3, since we can make the words 'eat' 'in' 'rat' without
#them touching each other. 
#We could have alternatively made 'eat' and 'rain',  but that would be incorrect since that's only 2 words.

In [ ]:
#Solution: backtracking

In [ ]:
DIRECTIONS = [
    (1, 0),
    (-1, 0),
    (0, 1),
    (0, -1),
]

def max_words(board, n, m, words, visited, r, c, curr_word):
    if r < 0 or r >= n or c < 0 or c >= m or visited[r][c]:
        return []
    
    curr_word += board[r][c]
    #if no words in |words| start with |curr_word|, then return early.
    if not any(word.startswith(curr_word) for word in words):
        return []
    
    visited[r][c] = True
    
    max_word_set = []
    if curr_word in words:
        #A valid words has been found: terminate current word search and start a new one
        for r, row in enumerate(board):
            for c, val in enumerate(row):
                curr_word_set = max_words(board, n, m, words, visited, r, c, '')
                if len(curr_word_set) > len(max_word_set):
                    max_word_set = curr_word_set
        max_word_set.append(curr_word)
    else:
        for dr, dc in DIRECTIONS:
            curr_word_set = max_words(board, n, m, words, visited, r + dr, c + dc, curr_word)
            if len(curr_word_set) > len(max_word_set):
                max_word_set = curr_word_set
                
    visited[r][c] = False
    return max_word_set

def find_max_words(board, words):
    if not board:
        return 0
    
    n, m = len(board), len(board[0])
    visited = [[False for _ in range(m)] for ]

##348

In [ ]:
#Zillow

#A ternary search tree is a trie-like data structure where each node may have up to three children.
#Here is an example which represents the words
#code, cob, be, ax, war, and we.

#c
#b     o     w
#a e   d     a
#x    b e   r e

#The tree is structured according to the following rules:
# - left child nodes link to words lexicographically earlier than the parent prefix
# - right child nodes link to words lexxicographically later than the parents prefix
# - middle child nodes continue the current word

#for instance, since code is the first word inserted in the tree, and cob lexicographically
#precedes cod, cob is represented as a left child extending from cod.

#implement insertion and search functions for a ternary search tree.

In [ ]:
#Begin by creating a class to represent a tree node,
#just as if we were creating a binary tree.
#In this case, we will declare three child nodes instead of two,
#and add an additional variable that tells us if we have reached the end of a word

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.mid = None
        self.right = None
        self.end = False
    
#If there are no nodes in our tree, the process is straightforward
#we keep addidg mid child nodes with each consecutive letter until we spell out the whole word
#and se the last child's end value to be True

#more complicated cases? -> recursive
#At each step, we will consider two object: the current node, and the remainder of the word
#that must be inserted. If the current node is None, we find ourselves in the case above.
#Otherwise, we can consider three possibilities.
# - If the first letter of the word remainder precedes the current node's letter, we branch left
# - If the first letter of the word remainder succeeds the current node's letter, we branch right
# - If the two are equal, we continue to the middle child node with the tail of our word.
#Since this recursive process applies to an arbitrary node and suffix, we will call it as a helper
#from our main insert function, which updates the root of our tree.

class TernaryTree:
    def __inin__(self):
        self.root = None
        
    def _insert(self, node, word):
        if not word:
            return node
        
        head, tail = word[0], word[1:]
        if not node:
            node = Node(head)
            
        if head < node.data:
            node.left = self._insert(node.left, word)
        elif head > node.data:
            node.right = self._insert(node.right, word)
        else:
            if not tail:
                node.end = True
            else:
                node.mid = self._insert(node.mid, tail)
                
        return node
    
    def insert(self, word):
        self.root = self._insert(self.root, word)
        
    #info

In [ ]:
class TernaryTree:
    def __init__(self):
        self.root = None
        
    ...
    
    def _search(self, node, word):
        if not node or not word:
            return False
        
        head, tail = word[0], word[1:]
        if head < node.data:
            return self._search(node.left, word)
        elif head > node.data:
            return self._search(node.right, word)
        else:
            if not tail and node.end:
                return True
            return self._search(node.mid, tail)
        
    def search(self, word):
        return self._search(self.root, word)
    
    